In [1]:
%reload_ext dotenv
%dotenv .env

In [2]:
import os
import sqlite3

from classes import TramStop, TramPassage, DayType, Departure

connection = sqlite3.connect(os.environ.get('DATABASE_NAME'))
cursor = connection.cursor()

In [3]:
cursor.execute("SELECT id, name, latitude, longitude FROM tram_stops")

tram_stops: dict[int, TramStop] = {
    item[0]: TramStop(*item) for item in cursor.fetchall()
}

tram_stops

{213578731: TramStop(node_id=213578731, name='Kombinat 01', latitude=50.0791355, longitude=20.0650412),
 213585855: TramStop(node_id=213585855, name='Cienista 01', latitude=50.0853912, longitude=20.0385224),
 213585864: TramStop(node_id=213585864, name='Teatr Ludowy 01', latitude=50.082406, longitude=20.0329435),
 213585881: TramStop(node_id=213585881, name='Rondo Kocmyrzowskie im. Ks. Gorzelanego 02', latitude=50.0790506, longitude=20.0264899),
 213585893: TramStop(node_id=213585893, name='Bieńczycka 01', latitude=50.0755265, longitude=20.0201847),
 213585922: TramStop(node_id=213585922, name='Stella-Sawickiego 02', latitude=50.0743017, longitude=20.0046383),
 213585931: TramStop(node_id=213585931, name='AWF / PK 02', latitude=50.0740094, longitude=19.9989789),
 213592007: TramStop(node_id=213592007, name='Teatr Variété 02', latitude=50.058832, longitude=19.9623647),
 213593804: TramStop(node_id=213593804, name='Ofiar Dąbia 02', latitude=50.0596288, longitude=19.9762713),
 213593828: 

# Line 1

In [4]:
day, line_id, line_variant = DayType.WEEKDAY, 1, 1

cursor.execute("""
    SELECT
        tram_stops.id,
        tram_departures.stop_index,
        tram_departures.hour,
        tram_departures.minute
    FROM tram_departures
        JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
        JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
    WHERE tram_departures.day = ? AND tram_line_variants.id = ?
    ORDER BY
        tram_departures.stop_index DESC,
        tram_departures.hour DESC,
        tram_departures.minute DESC
""", (day.value, line_variant))

departures = [Departure(item[0], line_variant, item[1], day, item[2], item[3]) for item in cursor.fetchall()]
departures

[Departure(stop_id=11876937946, variant_id=1, stop_index=29, day_type=<DayType.WEEKDAY: 'W'>, hour=23, minute=30),
 Departure(stop_id=11876937946, variant_id=1, stop_index=29, day_type=<DayType.WEEKDAY: 'W'>, hour=23, minute=10),
 Departure(stop_id=11876937946, variant_id=1, stop_index=29, day_type=<DayType.WEEKDAY: 'W'>, hour=22, minute=52),
 Departure(stop_id=11876937946, variant_id=1, stop_index=29, day_type=<DayType.WEEKDAY: 'W'>, hour=22, minute=31),
 Departure(stop_id=11876937946, variant_id=1, stop_index=29, day_type=<DayType.WEEKDAY: 'W'>, hour=22, minute=12),
 Departure(stop_id=11876937946, variant_id=1, stop_index=29, day_type=<DayType.WEEKDAY: 'W'>, hour=21, minute=53),
 Departure(stop_id=11876937946, variant_id=1, stop_index=29, day_type=<DayType.WEEKDAY: 'W'>, hour=21, minute=37),
 Departure(stop_id=11876937946, variant_id=1, stop_index=29, day_type=<DayType.WEEKDAY: 'W'>, hour=21, minute=18),
 Departure(stop_id=11876937946, variant_id=1, stop_index=29, day_type=<DayType.W

In [5]:
passages: list[TramPassage] = []
departure_passage: dict[int, TramPassage] = {}

for end_departure in filter(lambda x: x not in departure_passage, departures):
    passage = TramPassage(line_id, day)
    passages.append(passage)

    previous_stop = end_departure
    for item in filter(lambda x: x is end_departure or x.is_previous(previous_stop), departures):
        if item in departure_passage:
            later_passage = departure_passage[item]
            for _ in range(later_passage.get_departure_index(item), len(later_passage)):
                del departure_passage[later_passage.reverse_stop_sequence.pop()]

        passage.add_stop(item)
        departure_passage[item] = passage
        previous_stop = item

[item.get_stop_names(tram_stops) for item in passages]

[['Elektromontaż 02',
  'Grodzki Urząd Pracy 02',
  'Wańkowicza 01',
  'Cienista 01',
  'Teatr Ludowy 01',
  'Rondo Kocmyrzowskie im. Ks. Gorzelanego 02',
  'Bieńczycka 01',
  'Rondo Czyżyńskie 04',
  'Centralna 02',
  'Rondo 308. Dywizjonu 02',
  'Ogród Doświadczeń 02',
  'TAURON Arena Kraków al. Pokoju 02',
  'Dąbie 02',
  'Ofiar Dąbia 02',
  'Fabryczna 02',
  'Francesco Nullo 02',
  'Teatr Variété 02',
  'Rondo Grzegórzeckie 02',
  'Hala Targowa 02',
  'Starowiślna 04',
  'Poczta Główna 02',
  'Plac Wszystkich Świętych 01',
  'Filharmonia 01',
  'UJ / AST 01',
  'Muzeum Narodowe 01',
  'Oleandry 01',
  'Park Jordana 01',
  'Reymana 01',
  'Cichy Kącik'],
 ['Elektromontaż 02',
  'Grodzki Urząd Pracy 02',
  'Wańkowicza 01',
  'Cienista 01',
  'Teatr Ludowy 01',
  'Rondo Kocmyrzowskie im. Ks. Gorzelanego 02',
  'Bieńczycka 01',
  'Rondo Czyżyńskie 04',
  'Centralna 02',
  'Rondo 308. Dywizjonu 02',
  'Ogród Doświadczeń 02',
  'TAURON Arena Kraków al. Pokoju 02',
  'Dąbie 02',
  'Ofiar 

## Line 50

#### Borek Fałęcki → Górka Narodowa P+R

In [6]:
day, line_id, line_variant = DayType.WEEKDAY, 2, 6

cursor.execute("""
    SELECT
        tram_stops.id,
        tram_departures.stop_index,
        tram_departures.hour,
        tram_departures.minute
    FROM tram_departures
        JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
        JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
    WHERE tram_departures.day = ? AND tram_line_variants.id = ?
    ORDER BY
        tram_departures.stop_index DESC,
        tram_departures.hour DESC,
        tram_departures.minute DESC
""", (day.value, line_variant))

departures = [Departure(item[0], line_variant, item[1], day, item[2], item[3]) for item in cursor.fetchall()]
departures

[Departure(stop_id=11498423078, variant_id=6, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=23, minute=47),
 Departure(stop_id=11498423078, variant_id=6, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=23, minute=22),
 Departure(stop_id=11498423078, variant_id=6, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=23, minute=2),
 Departure(stop_id=11498423078, variant_id=6, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=22, minute=42),
 Departure(stop_id=11498423078, variant_id=6, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=22, minute=22),
 Departure(stop_id=11498423078, variant_id=6, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=22, minute=7),
 Departure(stop_id=11498423078, variant_id=6, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=21, minute=52),
 Departure(stop_id=11498423078, variant_id=6, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=21, minute=35),
 Departure(stop_id=11498423078, variant_id=6, stop_index=36, day_type=<DayType.WEE

In [7]:
passages: list[TramPassage] = []
departure_passage: dict[int, TramPassage] = {}

highest_stop_index = max(map(lambda x: x.stop_index, departures))
for end_departure in filter(lambda x: x not in departure_passage, departures):
    passage = TramPassage(line_id, day)
    passages.append(passage)

    if end_departure.stop_index < highest_stop_index:
        print("Second-last stop somewhere else:", tram_stops[end_departure.stop_id].name)

    previous_stop = end_departure
    for item in filter(lambda x: x is end_departure or x.is_previous(previous_stop), departures):
        if item in departure_passage:
            later_passage = departure_passage[item]
            for _ in range(later_passage.get_departure_index(item), len(later_passage)):
                del departure_passage[later_passage.reverse_stop_sequence.pop()]

        passage.add_stop(item)
        departure_passage[item] = passage
        previous_stop = item

[item.get_stop_names(tram_stops) for item in passages]

[['Borek Fałęcki 01',
  'Borek Fałęcki I 02',
  'Solvay 02',
  'Łagiewniki SKA 03',
  'Sanktuarium Bożego Miłosierdzia 01',
  'Turowicza 01',
  'Kurdwanów P+R 03',
  'Kurdwanów P+R 01',
  'Witosa 02',
  'Nowosądecka 02',
  'Piaski Nowe 02',
  'Dauna 02',
  'Bieżanowska 04',
  'Kabel 02',
  'Dworzec Płaszów Estakada 01',
  'Lipska 02',
  'Gromadzka 02',
  'Kuklińskiego 02',
  'Klimeckiego 02',
  'Zabłocie 02',
  'Rondo Grzegórzeckie 03',
  'Rondo Mogilskie 05',
  'Dworzec Główny Tunel 02',
  'Politechnika 03',
  'Dworzec Towarowy 01',
  'Szpital Narutowicza 01',
  'Bratysławska 01',
  'Krowodrza Górka P+R 01',
  'Pachońskiego P+R 01',
  'Białoprądnicka 02',
  'Górnickiego 02',
  'Siewna Wiadukt 01',
  'Bociana 01',
  'Kuźnicy Kołłątajowskiej 01',
  'Papierni Prądnickich 01',
  'Górka Narodowa P+R 05'],
 ['Borek Fałęcki 01',
  'Borek Fałęcki I 02',
  'Solvay 02',
  'Łagiewniki SKA 03',
  'Sanktuarium Bożego Miłosierdzia 01',
  'Turowicza 01',
  'Kurdwanów P+R 03',
  'Kurdwanów P+R 01',
 

#### Górka Narodowa P+R → Borek Fałęcki

In [8]:
day, line_id, line_variant = DayType.WEEKDAY, 2, 5

cursor.execute("""
    SELECT
        tram_stops.id,
        tram_departures.stop_index,
        tram_departures.hour,
        tram_departures.minute
    FROM tram_departures
        JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
        JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
    WHERE tram_departures.day = ? AND tram_line_variants.id = ?
    ORDER BY
        tram_departures.stop_index DESC,
        tram_departures.hour DESC,
        tram_departures.minute DESC
""", (day.value, line_variant))

departures = [Departure(item[0], line_variant, item[1], day, item[2], item[3]) for item in cursor.fetchall()]
departures

[Departure(stop_id=321429856, variant_id=5, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=23, minute=58),
 Departure(stop_id=321429856, variant_id=5, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=23, minute=38),
 Departure(stop_id=321429856, variant_id=5, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=23, minute=18),
 Departure(stop_id=321429856, variant_id=5, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=22, minute=58),
 Departure(stop_id=321429856, variant_id=5, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=22, minute=41),
 Departure(stop_id=321429856, variant_id=5, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=22, minute=26),
 Departure(stop_id=321429856, variant_id=5, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=22, minute=11),
 Departure(stop_id=321429856, variant_id=5, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour=21, minute=56),
 Departure(stop_id=321429856, variant_id=5, stop_index=36, day_type=<DayType.WEEKDAY: 'W'>, hour

In [9]:
passages: list[TramPassage] = []
departure_passage: dict[int, TramPassage] = {}

highest_stop_index = max(map(lambda x: x.stop_index, departures))
for end_departure in filter(lambda x: x not in departure_passage, departures):
    passage = TramPassage(line_id, day)
    passages.append(passage)

    if end_departure.stop_index < highest_stop_index:
        print("Second stop somewhere else:", tram_stops[end_departure.stop_id].name)

        cursor.execute("""
            SELECT DISTINCT tram_stops.id
            FROM tram_departures
                JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
                JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
            WHERE tram_departures.day = ? AND tram_line_variants.id = ? AND tram_departures.stop_index = ?
        """, (day.value, line_variant, end_departure.stop_index + 1,))

        new_stop_id = cursor.fetchone()[0]
        new_last_departure = Departure(
            new_stop_id,
            line_variant,
            end_departure.stop_index + 1,
            day,
            (end_departure.hour + (end_departure.minute + 1) // 60) % 24,
            (end_departure.minute + 1) % 60
        )

        departures.append(new_last_departure)
        passage.add_stop(new_last_departure)
        departure_passage[new_last_departure] = passage

    previous_stop = end_departure
    for item in filter(lambda x: x is end_departure or x.is_previous(previous_stop), departures):
        if item in departure_passage:
            later_passage = departure_passage[item]
            for _ in range(later_passage.get_departure_index(item), len(later_passage)):
                del departure_passage[later_passage.reverse_stop_sequence.pop()]

        passage.add_stop(item)
        departure_passage[item] = passage
        previous_stop = item

[item.get_stop_names(tram_stops) for item in passages]

Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop somewhere else: Witosa 01
Second stop 

[['Górka Narodowa P+R 01',
  'Papierni Prądnickich 02',
  'Kuźnicy Kołłątajowskiej 02',
  'Bociana 02',
  'Siewna Wiadukt 02',
  'Górnickiego 01',
  'Białoprądnicka 01',
  'Pachońskiego P+R 02',
  'Krowodrza Górka P+R 02',
  'Bratysławska 02',
  'Szpital Narutowicza 02',
  'Dworzec Towarowy 02',
  'Politechnika 04',
  'Dworzec Główny Tunel 01',
  'Rondo Mogilskie 03',
  'Rondo Grzegórzeckie 04',
  'Zabłocie 01',
  'Klimeckiego 01',
  'Kuklińskiego 01',
  'Gromadzka 01',
  'Lipska 03',
  'Dworzec Płaszów Estakada 02',
  'Kabel 01',
  'Bieżanowska 03',
  'Dauna 01',
  'Piaski Nowe 01',
  'Nowosądecka 01',
  'Witosa 01',
  'Kurdwanów P+R 08',
  'Kurdwanów P+R 10',
  'Turowicza 02',
  'Sanktuarium Bożego Miłosierdzia 02',
  'Łagiewniki SKA 01',
  'Solvay 01',
  'Borek Fałęcki I 01',
  'Borek Fałęcki'],
 ['Górka Narodowa P+R 01',
  'Papierni Prądnickich 02',
  'Kuźnicy Kołłątajowskiej 02',
  'Bociana 02',
  'Siewna Wiadukt 02',
  'Górnickiego 01',
  'Białoprądnicka 01',
  'Pachońskiego P+R 